## Import libraries

In [191]:
import numpy as np  
import pandas as pd

import re  
import nltk  
import fitz
import json
import pickle  
import string
import PyPDF2
import glob
import anvil.server
import anvil.media


from operator import itemgetter
from re import findall
from re import sub
from os import listdir
from os.path import isfile, join

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn import metrics

### Anvil Server

In [177]:
anvil.server.connect("M6N3N6W5QEEUP57BOJVTJL5A-ZEEMKNPO5KOQ7SMR")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


## Reading Resumes into DataFrame

Does it have education?
Does it have professional?
Does it have project experience?
Does it have professional associations?

This block reads every PDF file located in the folder. There are 81 resumes, so, anything after 81 will be labeled as **"not resume"**

In [42]:
# All the pdf files in the folder
pdf_files = glob.glob("%s/*.pdf" % "../resumes/")

i = 0 # 81 is the start of not resume
styles = {}
for file in pdf_files:
    doc = fitz.open(file)
    for page in doc:
        blocks = page.getText("dict")["blocks"]
        fonts = []
        sizes = []
        colors = []
        images = []
        body = ''
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        body += " " + s['text'].lower()
                        fonts.append(s['font'])
                        sizes.append(s['size'])
                        colors.append(s['color'])
            if b['type'] == 1:  # block contains image
                images.append(1)
        if 'education' in body:
            ed_check = 1
        else:
            ed_check = 0
        if 'professional registrations' in body:
            pr_check = 1
        else:
            pr_check = 0
        if 'associations' in body:
            pa_check = 1
        else:
            pa_check = 0
        if 'experience' in body:
            pe_check = 1
        else:
            pe_check = 0
        if len(images)>0:
            img_check = 1
        else:
            img_check = 0
        
    if i < 81:
        label = 'resume'
    else:
        label = 'not resume'
    styles[doc] =  {'name':doc, 'body':body,'fonts':list(set(fonts)), 'size':list(set(sizes)),'color':list(set(colors)),'education':ed_check, 'registrations':pr_check,
                   'associations':pa_check, 'experience':pe_check, 'image':img_check, 'label':label}
    i = i + 1

data = pd.DataFrame.from_dict(styles, orient='index')

mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
m

In [129]:
df = pd.DataFrame.from_dict(styles, orient='columns')
data = df.transpose()

In [239]:
@anvil.server.callable
def read_text(pdf):
    styles = {}
    print('Started')
    with anvil.media.TempFile(pdf) as f:
        doc = fitz.open(f)
        for page in doc:
            blocks = page.getText("dict")["blocks"]
            fonts = []
            sizes = []
            colors = []
            images = []
            body = ''            
            for b in blocks:  # iterate through the text blocks
                if b['type'] == 0:  # block contains text
                    for l in b["lines"]:  # iterate through the text lines
                        for s in l["spans"]:  # iterate through the text spans
                            body += " " + s['text'].lower()
                            fonts.append(s['font'])
                            sizes.append(s['size'])
                            colors.append(s['color'])
                if b['type'] == 1:  # block contains image
                    images.append(1)
            if 'education' in body:
                ed_check = 1
            else:
                ed_check = 0
            if 'professional registrations' in body:
                pr_check = 1
            else:
                pr_check = 0
            if 'associations' in body:
                pa_check = 1
            else:
                pa_check = 0
            if 'experience' in body:
                pe_check = 1
            else:
                pe_check = 0
            if len(images)>0:
                img_check = 1
            else:
                img_check = 0

        if i < 81:
            label = 'resume'
        else:
            label = 'not resume'
        styles[doc] =  {'name':doc, 'body':body,'fonts':list(set(fonts)), 'size':list(set(sizes)),'color':list(set(colors)),'education':ed_check, 'registrations':pr_check,
                       'associations':pa_check, 'experience':pe_check, 'image':img_check, 'label':label}
        df = pd.DataFrame.from_dict(styles, orient='columns')
        data = df.transpose()
        
        resp = pipe.predict(data)[0]
        return resp

Started
Started
Started
Started
Started


### Text and Numeric Tranformers
If there is a dtype error, change the type of the transformer. Object and float64 do not match.

In [198]:
# text and numeric classes that use sklearn base libaries
class TextTransformer(BaseEstimator, TransformerMixin):
    """
    Transform text features
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None, *parg, **kwarg):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberTransformer(BaseEstimator, TransformerMixin):
    """
    Transform numeric features
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]].astype(float)

### Model Training

In [199]:
df = data
from sklearn.feature_extraction.text import TfidfTransformer
vec_tdidf = TfidfVectorizer(ngram_range=(1,1), analyzer='word', norm='l2')

# compile both the TextTransformer and TfidfVectorizer 
# to the text 'Text_Feature' 
body_text = Pipeline([
                ('transformer', TextTransformer(key='body')),
                ('vectorizer', vec_tdidf)
                ])

ed_numeric = Pipeline([
                ('transformer', NumberTransformer(key='education')),
                ])
reg_numeric = Pipeline([
                ('transformer', NumberTransformer(key='registrations')),
                ])
assc_numeric = Pipeline([
                ('transformer', NumberTransformer(key='associations')),
                ])
exp_numeric = Pipeline([
                ('transformer', NumberTransformer(key='experience')),
                ])
img_numeric = Pipeline([
                ('transformer', NumberTransformer(key='image')),
                ])

# combine all of the features, text and numeric together
features = FeatureUnion([ ('body', body_text),
                          ('education', ed_numeric),
                          ('registrations', reg_numeric),
                          ('associations', assc_numeric),
                          ('experience', exp_numeric),
                          ('image', img_numeric)
                      ])

# create the classfier from RF
clf = RandomForestClassifier(max_depth=25, max_features = 0.2, n_estimators = 500 )

# unite the features and classfier together
pipe = Pipeline([('features', features),
                 ('clf',clf)
                 ])

# transform the categorical predictor into numeric
predicted_dummies = pd.get_dummies(df['label'])

# split the data into train and test
# isolate the features from the predicted field
text_numeric_features = ['body','education', 'registrations', 'associations', 'experience','image']
predictor = 'label'

X_train, X_test, y_train, y_test = train_test_split(df[text_numeric_features], df[predictor], 
                                                    test_size=0.25, random_state=42)

# fit the model
pipe.fit(X_train, y_train)

# predict from the test set
preds = pipe.predict(X_test)

# print out your accuracy
print("Accuracy:",metrics.accuracy_score(y_test, preds))

Accuracy: 0.9142857142857143


### Hyperparameter Tuning

In [175]:
# Hyperparameters
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV


param_grid = {
    "clf__n_estimators": [100, 500, 1000],
    "clf__max_depth": [1, 5, 10, 25],
    "clf__max_features": [*np.arange(0.1, 1.1, 0.1)],
}

clf = RandomForestClassifier()

X_train, X_test, y_train, y_test = train_test_split(df[text_numeric_features], df[predictor], 
                                                    test_size=0.25, random_state=42)

# unite the features and classfier together
pipe = Pipeline([('features', features),
                 ('clf',clf)
                 ])

# fit the model
pipe.fit(X_train, y_train)

# predict from the test set
preds = pipe.predict(X_test)

# print out your accuracy
print("Accuracy:",metrics.accuracy_score(y_test, preds))

# Perform grid search, fit it, and print score
gs = GridSearchCV(pipe, param_grid=param_grid, cv=3, n_jobs=1, verbose=1000) #n_jobs needs to be one because it cannot be pickled
gs.fit(X_train, y_train)


cv_results = pd.concat([pd.DataFrame(gs.cv_results_["params"]),pd.DataFrame(gs.cv_results_["mean_test_score"], columns=["Accuracy"])],axis=1)
cv_results.sort_values(by='Accuracy')


Accuracy: 0.9142857142857143
Fitting 3 folds for each of 120 candidates, totalling 360 fits
[CV 1/3; 1/120] START clf__max_depth=1, clf__max_features=0.1, clf__n_estimators=100
[CV 1/3; 1/120] END clf__max_depth=1, clf__max_features=0.1, clf__n_estimators=100;, score=0.794 total time=   0.0s
[CV 2/3; 1/120] START clf__max_depth=1, clf__max_features=0.1, clf__n_estimators=100
[CV 2/3; 1/120] END clf__max_depth=1, clf__max_features=0.1, clf__n_estimators=100;, score=0.912 total time=   0.1s
[CV 3/3; 1/120] START clf__max_depth=1, clf__max_features=0.1, clf__n_estimators=100
[CV 3/3; 1/120] END clf__max_depth=1, clf__max_features=0.1, clf__n_estimators=100;, score=0.941 total time=   0.0s
[CV 1/3; 2/120] START clf__max_depth=1, clf__max_features=0.1, clf__n_estimators=500
[CV 1/3; 2/120] END clf__max_depth=1, clf__max_features=0.1, clf__n_estimators=500;, score=0.824 total time=   0.5s
[CV 2/3; 2/120] START clf__max_depth=1, clf__max_features=0.1, clf__n_estimators=500
[CV 2/3; 2/120] END

[CV 1/3; 14/120] END clf__max_depth=1, clf__max_features=0.5, clf__n_estimators=500;, score=0.794 total time=   0.6s
[CV 2/3; 14/120] START clf__max_depth=1, clf__max_features=0.5, clf__n_estimators=500
[CV 2/3; 14/120] END clf__max_depth=1, clf__max_features=0.5, clf__n_estimators=500;, score=0.853 total time=   0.7s
[CV 3/3; 14/120] START clf__max_depth=1, clf__max_features=0.5, clf__n_estimators=500
[CV 3/3; 14/120] END clf__max_depth=1, clf__max_features=0.5, clf__n_estimators=500;, score=0.912 total time=   0.7s
[CV 1/3; 15/120] START clf__max_depth=1, clf__max_features=0.5, clf__n_estimators=1000
[CV 1/3; 15/120] END clf__max_depth=1, clf__max_features=0.5, clf__n_estimators=1000;, score=0.794 total time=   1.3s
[CV 2/3; 15/120] START clf__max_depth=1, clf__max_features=0.5, clf__n_estimators=1000
[CV 2/3; 15/120] END clf__max_depth=1, clf__max_features=0.5, clf__n_estimators=1000;, score=0.853 total time=   1.3s
[CV 3/3; 15/120] START clf__max_depth=1, clf__max_features=0.5, clf

[CV 1/3; 27/120] END clf__max_depth=1, clf__max_features=0.9, clf__n_estimators=1000;, score=0.794 total time=   1.7s
[CV 2/3; 27/120] START clf__max_depth=1, clf__max_features=0.9, clf__n_estimators=1000
[CV 2/3; 27/120] END clf__max_depth=1, clf__max_features=0.9, clf__n_estimators=1000;, score=0.853 total time=   1.6s
[CV 3/3; 27/120] START clf__max_depth=1, clf__max_features=0.9, clf__n_estimators=1000
[CV 3/3; 27/120] END clf__max_depth=1, clf__max_features=0.9, clf__n_estimators=1000;, score=0.912 total time=   1.7s
[CV 1/3; 28/120] START clf__max_depth=1, clf__max_features=1.0, clf__n_estimators=100
[CV 1/3; 28/120] END clf__max_depth=1, clf__max_features=1.0, clf__n_estimators=100;, score=0.794 total time=   0.1s
[CV 2/3; 28/120] START clf__max_depth=1, clf__max_features=1.0, clf__n_estimators=100
[CV 2/3; 28/120] END clf__max_depth=1, clf__max_features=1.0, clf__n_estimators=100;, score=0.853 total time=   0.1s
[CV 3/3; 28/120] START clf__max_depth=1, clf__max_features=1.0, cl

[CV 1/3; 40/120] END clf__max_depth=5, clf__max_features=0.4, clf__n_estimators=100;, score=0.824 total time=   0.1s
[CV 2/3; 40/120] START clf__max_depth=5, clf__max_features=0.4, clf__n_estimators=100
[CV 2/3; 40/120] END clf__max_depth=5, clf__max_features=0.4, clf__n_estimators=100;, score=0.971 total time=   0.1s
[CV 3/3; 40/120] START clf__max_depth=5, clf__max_features=0.4, clf__n_estimators=100
[CV 3/3; 40/120] END clf__max_depth=5, clf__max_features=0.4, clf__n_estimators=100;, score=1.000 total time=   0.1s
[CV 1/3; 41/120] START clf__max_depth=5, clf__max_features=0.4, clf__n_estimators=500
[CV 1/3; 41/120] END clf__max_depth=5, clf__max_features=0.4, clf__n_estimators=500;, score=0.824 total time=   0.8s
[CV 2/3; 41/120] START clf__max_depth=5, clf__max_features=0.4, clf__n_estimators=500
[CV 2/3; 41/120] END clf__max_depth=5, clf__max_features=0.4, clf__n_estimators=500;, score=0.941 total time=   0.9s
[CV 3/3; 41/120] START clf__max_depth=5, clf__max_features=0.4, clf__n_

[CV 1/3; 53/120] END clf__max_depth=5, clf__max_features=0.8, clf__n_estimators=500;, score=0.824 total time=   1.1s
[CV 2/3; 53/120] START clf__max_depth=5, clf__max_features=0.8, clf__n_estimators=500
[CV 2/3; 53/120] END clf__max_depth=5, clf__max_features=0.8, clf__n_estimators=500;, score=0.971 total time=   1.2s
[CV 3/3; 53/120] START clf__max_depth=5, clf__max_features=0.8, clf__n_estimators=500
[CV 3/3; 53/120] END clf__max_depth=5, clf__max_features=0.8, clf__n_estimators=500;, score=1.000 total time=   1.2s
[CV 1/3; 54/120] START clf__max_depth=5, clf__max_features=0.8, clf__n_estimators=1000
[CV 1/3; 54/120] END clf__max_depth=5, clf__max_features=0.8, clf__n_estimators=1000;, score=0.824 total time=   2.3s
[CV 2/3; 54/120] START clf__max_depth=5, clf__max_features=0.8, clf__n_estimators=1000
[CV 2/3; 54/120] END clf__max_depth=5, clf__max_features=0.8, clf__n_estimators=1000;, score=0.941 total time=   2.6s
[CV 3/3; 54/120] START clf__max_depth=5, clf__max_features=0.8, clf

...

[CV 3/3; 79/120] END clf__max_depth=10, clf__max_features=0.7000000000000001, clf__n_estimators=100;, score=1.000 total time=   0.2s
[CV 1/3; 80/120] START clf__max_depth=10, clf__max_features=0.7000000000000001, clf__n_estimators=500
[CV 1/3; 80/120] END clf__max_depth=10, clf__max_features=0.7000000000000001, clf__n_estimators=500;, score=0.824 total time=   1.0s
[CV 2/3; 80/120] START clf__max_depth=10, clf__max_features=0.7000000000000001, clf__n_estimators=500
[CV 2/3; 80/120] END clf__max_depth=10, clf__max_features=0.7000000000000001, clf__n_estimators=500;, score=0.971 total time=   1.1s
[CV 3/3; 80/120] START clf__max_depth=10, clf__max_features=0.7000000000000001, clf__n_estimators=500
[CV 3/3; 80/120] END clf__max_depth=10, clf__max_features=0.7000000000000001, clf__n_estimators=500;, score=0.971 total time=   1.1s
[CV 1/3; 81/120] START clf__max_depth=10, clf__max_features=0.7000000000000001, clf__n_estimators=1000
[CV 1/3; 81/120] END clf__max_depth=10, clf__max_features=0

[CV 3/3; 92/120] END clf__max_depth=25, clf__max_features=0.1, clf__n_estimators=500;, score=1.000 total time=   0.6s
[CV 1/3; 93/120] START clf__max_depth=25, clf__max_features=0.1, clf__n_estimators=1000
[CV 1/3; 93/120] END clf__max_depth=25, clf__max_features=0.1, clf__n_estimators=1000;, score=0.824 total time=   1.2s
[CV 2/3; 93/120] START clf__max_depth=25, clf__max_features=0.1, clf__n_estimators=1000
[CV 2/3; 93/120] END clf__max_depth=25, clf__max_features=0.1, clf__n_estimators=1000;, score=1.000 total time=   1.3s
[CV 3/3; 93/120] START clf__max_depth=25, clf__max_features=0.1, clf__n_estimators=1000
[CV 3/3; 93/120] END clf__max_depth=25, clf__max_features=0.1, clf__n_estimators=1000;, score=1.000 total time=   1.3s
[CV 1/3; 94/120] START clf__max_depth=25, clf__max_features=0.2, clf__n_estimators=100
[CV 1/3; 94/120] END clf__max_depth=25, clf__max_features=0.2, clf__n_estimators=100;, score=0.824 total time=   0.1s
[CV 2/3; 94/120] START clf__max_depth=25, clf__max_featu

[CV 3/3; 105/120] END clf__max_depth=25, clf__max_features=0.5, clf__n_estimators=1000;, score=1.000 total time=   2.0s
[CV 1/3; 106/120] START clf__max_depth=25, clf__max_features=0.6, clf__n_estimators=100
[CV 1/3; 106/120] END clf__max_depth=25, clf__max_features=0.6, clf__n_estimators=100;, score=0.824 total time=   0.1s
[CV 2/3; 106/120] START clf__max_depth=25, clf__max_features=0.6, clf__n_estimators=100
[CV 2/3; 106/120] END clf__max_depth=25, clf__max_features=0.6, clf__n_estimators=100;, score=0.941 total time=   0.2s
[CV 3/3; 106/120] START clf__max_depth=25, clf__max_features=0.6, clf__n_estimators=100
[CV 3/3; 106/120] END clf__max_depth=25, clf__max_features=0.6, clf__n_estimators=100;, score=0.971 total time=   0.2s
[CV 1/3; 107/120] START clf__max_depth=25, clf__max_features=0.6, clf__n_estimators=500
[CV 1/3; 107/120] END clf__max_depth=25, clf__max_features=0.6, clf__n_estimators=500;, score=0.824 total time=   1.0s
[CV 2/3; 107/120] START clf__max_depth=25, clf__max_

[CV 3/3; 118/120] END clf__max_depth=25, clf__max_features=1.0, clf__n_estimators=100;, score=0.941 total time=   0.2s
[CV 1/3; 119/120] START clf__max_depth=25, clf__max_features=1.0, clf__n_estimators=500
[CV 1/3; 119/120] END clf__max_depth=25, clf__max_features=1.0, clf__n_estimators=500;, score=0.824 total time=   1.2s
[CV 2/3; 119/120] START clf__max_depth=25, clf__max_features=1.0, clf__n_estimators=500
[CV 2/3; 119/120] END clf__max_depth=25, clf__max_features=1.0, clf__n_estimators=500;, score=0.941 total time=   1.4s
[CV 3/3; 119/120] START clf__max_depth=25, clf__max_features=1.0, clf__n_estimators=500
[CV 3/3; 119/120] END clf__max_depth=25, clf__max_features=1.0, clf__n_estimators=500;, score=1.000 total time=   1.4s
[CV 1/3; 120/120] START clf__max_depth=25, clf__max_features=1.0, clf__n_estimators=1000
[CV 1/3; 120/120] END clf__max_depth=25, clf__max_features=1.0, clf__n_estimators=1000;, score=0.824 total time=   2.6s
[CV 2/3; 120/120] START clf__max_depth=25, clf__max

,clf__max_depth,clf__max_features,clf__n_estimators,Accuracy
21,1,0.8,100,0.843137
28,1,1.0,500,0.852941
26,1,0.9,1000,0.852941
25,1,0.9,500,0.852941
24,1,0.9,100,0.852941
...,...,...,...,...
92,25,0.1,1000,0.941176
95,25,0.2,1000,0.941176
96,25,0.3,100,0.941176
31,5,0.1,500,0.941176


### Read Actual PDF Data


In [33]:
# All the pdf files in the folder
pdf_files = glob.glob("%s/*.pdf" % "../resumesnot/")

i = 0 # 81 is the start of not resume
styles = {}
for file in pdf_files:
    doc = fitz.open(file)
    for page in doc:
        blocks = page.getText("dict")["blocks"]
        fonts = []
        sizes = []
        colors = []
        images = []
        body = ''
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        body += " " + s['text'].lower()
                        fonts.append(s['font'])
                        sizes.append(s['size'])
                        colors.append(s['color'])
            if b['type'] == 1:  # block contains image
                images.append(1)
        if 'education' in body:
            ed_check = 1
        else:
            ed_check = 0
        if 'professional registrations' in body:
            pr_check = 1
        else:
            pr_check = 0
        if 'associations' in body:
            pa_check = 1
        else:
            pa_check = 0
        if 'experience' in body:
            pe_check = 1
        else:
            pe_check = 0
        if len(images)>0:
            img_check = 1
        else:
            img_check = 0

        styles[page] =  {'name':doc, 'body':body,'fonts':list(set(fonts)), 'size':list(set(sizes)),'color':list(set(colors)),'education':ed_check, 'registrations':pr_check,
                   'associations':pa_check, 'experience':pe_check, 'image':img_check}
    i = i + 1

pdf_data = pd.DataFrame.from_dict(styles, orient='index')

In [34]:
# Prediction using actual proposal data
preds = pipe.predict(pdf_data)

In [35]:
# Creating a dataframe with predicted labels
pdf_data['preds'] = preds
pdf_data[pdf_data['preds'] == 'resume']

,name,body,fonts,size,color,education,registrations,associations,experience,image,preds
page 11 of ../resumesnot\19.00150_Town of Vail_Gore Valley Trail_Feb19_FINAL_LR.pdf,(page 0 of ../resumesnot\19.00150_Town of Vail...,town of vail / gore valley trail design imp...,"[NewsGothicStd-Bold, NewsGothicStd]","[8.0, 10.0, 11.0, 12.0]","[0, 32117, 16777215]",0,0,0,1,1,resume
page 13 of ../resumesnot\20.02564 Grand Forks - East Grand Forks Traffic Impact Study - Final.pdf,(page 0 of ../resumesnot\20.02564 Grand Forks ...,section d: description of similar project...,"[NewsGothicStd-Bold, ArialNarrow]","[16.0, 10.5, 12.0]","[2373188, 13027530, 16296473, 811870, 2236191]",0,0,0,1,1,resume
page 15 of ../resumesnot\20.02564 Grand Forks - East Grand Forks Traffic Impact Study - Final.pdf,(page 0 of ../resumesnot\20.02564 Grand Forks ...,aaron lauinger principal-in-charge | authoriz...,"[ArialNarrow-Bold, NewsGothicStd-Bold, ArialNa...","[10.5, 10.5600004196167, 11.0, 12.0]","[16777215, 32117, 2236191]",0,0,0,1,1,resume
page 16 of ../resumesnot\20.02564 Grand Forks - East Grand Forks Traffic Impact Study - Final.pdf,(page 0 of ../resumesnot\20.02564 Grand Forks ...,key staff bios & project team resumes abdulla...,"[ArialNarrow-Bold, NewsGothicStd-Bold, ArialNa...","[10.5, 12.0]","[16777215, 2236191]",0,0,0,1,1,resume
page 17 of ../resumesnot\20.02564 Grand Forks - East Grand Forks Traffic Impact Study - Final.pdf,(page 0 of ../resumesnot\20.02564 Grand Forks ...,"will kerns, aicp | senior transportation pla...","[CIDFont+F3, CIDFont+F2, ArialNarrow, CIDFont+F4]","[8.982749938964844, 9.940999984741211, 8.98274...","[0, 2576241, 16777215]",1,1,1,1,1,resume
page 18 of ../resumesnot\20.02564 Grand Forks - East Grand Forks Traffic Impact Study - Final.pdf,(page 0 of ../resumesnot\20.02564 Grand Forks ...,"will kerns, aicp | senior transportation pla...","[CIDFont+F3, CIDFont+F4, ArialNarrow, CIDFont+...","[8.982749938964844, 9.94124984741211, 10.02000...","[0, 2576241]",1,0,0,0,1,resume
page 19 of ../resumesnot\20.02564 Grand Forks - East Grand Forks Traffic Impact Study - Final.pdf,(page 0 of ../resumesnot\20.02564 Grand Forks ...,jennifer turnbow | senior technical manager...,"[ArialNarrow-Bold, SymbolMT, ArialNarrow]","[9.0, 10.020000457763672, 19.979999542236328, ...","[0, 2576241, 16777215]",1,0,1,1,1,resume
page 20 of ../resumesnot\20.02564 Grand Forks - East Grand Forks Traffic Impact Study - Final.pdf,(page 0 of ../resumesnot\20.02564 Grand Forks ...,jennifer turnbow | senior technical manager...,"[ArialNarrow-Bold, SymbolMT, ArialNarrow]","[9.0, 10.020000457763672]","[0, 2576241]",0,0,0,0,1,resume
page 21 of ../resumesnot\20.02564 Grand Forks - East Grand Forks Traffic Impact Study - Final.pdf,(page 0 of ../resumesnot\20.02564 Grand Forks ...,"abdullah kurkcu, phd | lead traffic modele...","[ArialNarrow-Bold, Calibri, SymbolMT, ArialNar...","[9.0, 10.020000457763672, 10.979999542236328, ...","[0, 2576241, 16777215]",1,0,0,1,1,resume
page 22 of ../resumesnot\20.02564 Grand Forks - East Grand Forks Traffic Impact Study - Final.pdf,(page 0 of ../resumesnot\20.02564 Grand Forks ...,"abdullah kurkcu, phd | lead traffic modele...","[ArialNarrow-Bold, Calibri, SymbolMT, ArialNar...","[9.0, 10.979999542236328, 10.020000457763672]","[0, 2576241, 16777215]",0,0,1,1,1,resume


#### Notes
Since it transforms fonts and sizes to columns, new documents will not have the same amount of columns that the test data will have

In [113]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB

# Using Decision Tree Classifier on the data
dtclf = tree.DecisionTreeClassifier()
dtclf = dtclf.fit(resumes_train, y_train)
print(dtclf.score(resumes_test, y_test))


# Using Random Forest Classifier on the data
rfclf = RandomForestClassifier()
rfclf = rfclf.fit(resumes_train, y_train)
print(rfclf.score(resumes_test, y_test))

# Using SVM Classifier on the data
model_svm = svm.SVC()
model_svm = model_svm.fit(resumes_train, y_train)
print(model_svm.score(resumes_test, y_test))

# Using Bernoulli Naive Bayes Algorithm
bnbclf = BernoulliNB()
bnbclf = bnbclf.fit(resumes_train, y_train)
print(bnbclf.score(resumes_test, y_test))

# Using Gaussian Naive Bayes Algorithm
gnbclf = GaussianNB()
gnbclf = gnbclf.fit(resumes_train, y_train)
print(gnbclf.score(resumes_test, y_test))

0.9743589743589743
1.0
0.9743589743589743
0.9743589743589743
1.0
